#### seoul 따릉이 user prediction

### link

https://dacon.io/competitions/open/235576/overview/description

last update :  2023/01/28 0003i  
rank : 1465  
points : 121.29409  

In [39]:
import torch
import numpy as np


data = np.genfromtxt("C:/Users/OWO/Documents/AA_CODE/@Projects/Projects/@CONTEST/ct_dacon_0001_따릉이/train.csv",delimiter=',')



In [40]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'

In [41]:
#making list with no nan
data_nonan = []

for row in range(len(data)):
  flag = 0
  for col in range(11):
    if (np.isnan(data[row][col])) == True:
      flag = 1
      break
  if flag == 0:
    data_nonan.append(data[row])

In [4]:
for row in range(len(data_nonan)):#recheck
  for col in range(11):
    if (np.isnan(data_nonan[row][col])) == True:
      print("nan at %d"%row)

In [5]:
data_nonan = np.array(data_nonan)

In [6]:
len(data_nonan)

1328

In [7]:
x_train_list = np.array(data_nonan[1:700,1:-1]) #get 999 data leaving about 500 data left. that 500 data will be used as test data. in col we drop 1 == id and -1 count
y_train_list = np.array(data_nonan[1:700,-1:])

x_atest_list = np.array(data_nonan[700:,1:-1])
y_atest_list = np.array(data_nonan[700:,-1:])

In [8]:
x_train_tensor = torch.FloatTensor(x_train_list).to(device)
y_train_tensor = torch.FloatTensor(y_train_list).to(device)
x_atest_tensor = torch.FloatTensor(x_atest_list).to(device)
y_atest_tensor = torch.FloatTensor(y_atest_list).to(device)

In [9]:
len(x_atest_list)

628

In [10]:
len(y_atest_list)

628

In [11]:
len(x_train_tensor[0])

9

In [12]:
nb_epoch = 1000
dp = 0.1


In [13]:
#lr_list = [10,1,1e-1,1e-2,1e-3,1e-5,1e-7]
lr_list = [1e-3]
for LR in lr_list:
  print(LR)
  l1 = torch.nn.Linear(9,500)
  l2 = torch.nn.Linear(500,500)
  l3 = torch.nn.Linear(500,500)
  l4 = torch.nn.Linear(500,1)
  
  torch.nn.init.xavier_normal_(l1.weight)
  torch.nn.init.xavier_normal_(l2.weight)
  torch.nn.init.xavier_normal_(l3.weight)
  torch.nn.init.xavier_normal_(l4.weight)
  
  relu = torch.nn.ReLU()
  loss = torch.nn.MSELoss()
  drop = torch.nn.Dropout(p = dp)
  
  model = torch.nn.Sequential(l1,relu,drop,
                              l2,relu,drop,
                              l3,relu,drop,
                              l4).to(device)
  optimizer = torch.optim.Adam(model.parameters(),lr=LR)
  
  model.train()
  from tqdm import tqdm
  for epoch in tqdm(range(nb_epoch+1)):
    hypothesis = model(x_train_tensor)
    cost = loss(y_train_tensor,hypothesis)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    #if epoch % 200 == 0:
  print(cost.item())

0.001


100%|██████████| 1001/1001 [00:15<00:00, 64.31it/s]

2777.6279296875


In [14]:
#evaluate
model.eval()#you need this if you are using dropout

hypothesis = model(x_atest_tensor)
diff = hypothesis - y_atest_tensor
print(diff.sum()/len(x_atest_list))

tensor(-6.4595, grad_fn=<DivBackward0>)


In [15]:
submit_data = np.genfromtxt("C:/Users/OWO/Documents/AA_CODE/@Projects/Projects/@CONTEST/ct_dacon_0001_따릉이/test.csv",delimiter=',')



#making list with no nan
data_nonan = []
#put 1 in nan
for row in range(len(submit_data)):
  for col in range(10):
    if (np.isnan(submit_data[row][col])) == True:
        submit_data[row][col] = 1


In [16]:
submit_data

array([[1.000e+00, 1.000e+00, 1.000e+00, ..., 1.000e+00, 1.000e+00,
        1.000e+00],
       [0.000e+00, 7.000e+00, 2.070e+01, ..., 4.100e-02, 4.400e+01,
        2.700e+01],
       [1.000e+00, 1.700e+01, 3.000e+01, ..., 6.100e-02, 4.900e+01,
        3.600e+01],
       ...,
       [2.165e+03, 9.000e+00, 2.330e+01, ..., 2.000e-02, 1.700e+01,
        1.500e+01],
       [2.166e+03, 1.600e+01, 2.700e+01, ..., 3.200e-02, 4.000e+01,
        2.600e+01],
       [2.177e+03, 8.000e+00, 2.230e+01, ..., 7.000e-03, 3.000e+01,
        2.400e+01]])

In [17]:
for row in range(len(submit_data)):#recheck
  for col in range(len(submit_data[0])):
    if (np.isnan(submit_data[row][col])) == True:
      print("nan at %d"%row)

In [18]:
submit_data = np.array(submit_data[:,1:])

In [19]:
submit_data_tensor = torch.Tensor(submit_data)

In [20]:
hypothesis = model(submit_data_tensor)

In [37]:
submit_data = np.genfromtxt("C:/Users/OWO/Documents/AA_CODE/@Projects/@temp/4/test.csv",delimiter=',')
sub = []

for row in range(len(hypothesis)-1):
  temp = [submit_data[row+1][0],int(hypothesis.tolist()[row][0])]
  sub.append(temp)
  


# Insert the headers as the first row of the data array


print(sub[0])

[0.0, 2]


In [38]:
import numpy
numpy.savetxt('C:/Users/OWO/Documents/AA_CODE/@Projects/Projects/@CONTEST/ct_dacon_0001_따릉이/submission_temp.csv',sub,delimiter=",",fmt='%d')